In [1]:
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

In [40]:
for i in range(12):
	img = np.asarray(Image.open(f"lab/images/pos-{i}.tif"))

	a = img[:, :, 0]
	ind = np.unravel_index(np.argmax(a, axis=None), a.shape)

	fig, ax = plt.subplots()
	ax.plot(img[:, ind[1], 0])
	plt.savefig(f"lab/plots/plot-{i}.png")
	plt.close()